# Initial Workflow

In [4]:
# Importing Libraries
import os
import re
from dotenv import load_dotenv

# LangChain Core
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LLM Models
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic

# Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# RAGAS Evaluation Framework
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import (
    AspectCritic,
    RubricsScore,
    ContextPrecision,
    Faithfulness
)
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset


# Load environment variables from .env file
load_dotenv()


True

In [5]:
# Important: Please set your own password and database schema name from .env file
db_user = os.environ.get("DB_USERNAME") # It is usually 'root' for local MySQL installations
db_password = os.environ.get("DB_PASSWORD") # Set during MySQL installation


# Connect MySQL database
host = 'localhost'
port = '3306'
username = 'root'
password = db_password
database_schema = 'text_to_sql'
#mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
mysql_uri = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=2)

In [6]:
# Create the LLM Prompt Template

template = """Based on the table schema below, write a SQL query that would answer the user's question:
Remember : Only provide me the sql query do not include anything else. Provide me sql query in a single line do not add line breaks.
Table Schema: {schema}
Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [7]:
# get the schema of the database
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [8]:
# LLM Initialization
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = os.environ.get("GOOGLE_API_KEY"))

In [9]:
# Create the SQL query chain using the LLM and the prompt template

sql_chain = (
    RunnablePassthrough.assign(schema=lambda _:get_schema(db)) 
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)



In [10]:
# test the SQL query chain with a sample question

resp = sql_chain.invoke({"question": "What is the total 'Line Total' for Geiss Company"})
print(resp)

```sql
SELECT sum(T1.`Line Total`) FROM sales_order AS T1 INNER JOIN customers AS T2 ON T1.`Customer Name Index` = T2.`Customer Index` WHERE T2.`Customer Names` = "Geiss Company"
```


In [11]:
query = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE) # For removing ```sql ``` from the response

if query:
    query=query.group(1).strip()

query

'SELECT sum(T1.`Line Total`) FROM sales_order AS T1 INNER JOIN customers AS T2 ON T1.`Customer Name Index` = T2.`Customer Index` WHERE T2.`Customer Names` = "Geiss Company"'

In [12]:
# Execute the generated SQL query on the database
print(db.run(query))

[(5516846.999999994,)]


The generated answer matches our result when the same SQL query is run on MySQL Workbench